In [1]:
%gui qt5
%matplotlib qt5
from functools import partial as _partial
import numpy as np
from scipy import optimize

import matplotlib.pyplot as plt
import matplotlib.gridspec as mpl_gs
import matplotlib.cm as cmap
from matplotlib import rcParams
rcParams.update(
    {'font.size': 16,'lines.linewidth': 2, 'axes.grid': True})

import pyaccel
import mathphys
from pymodels import si

In [2]:
ring = si.create_accelerator()
ring.vchamber_on = True
famdata = si.get_family_data(ring)
bpms = np.array(famdata['BPM']['index']).flatten()
twi, *_ = pyaccel.optics.calc_twiss(ring)
bpmpos = twi.spos[bpms]
betx = twi.betax[bpms]
bety = twi.betay[bpms]
etax = twi.etax[bpms]
etax_ave = np.mean(etax)  # average of dispersion function at bpms
etay = twi.etay[bpms]
mux = twi.mux[bpms]/2/np.pi
muy = twi.muy[bpms]/2/np.pi

In [119]:
def _apply_linearxy(x_uncal, y_uncal):
    """."""
    K = 12e-3/np.sqrt(2)/np.sin(1/2)*1/2
    x_cal = x_uncal * K
    y_cal = y_uncal * K
    return x_cal, y_cal


def _apply_polyxy(x_uncal, y_uncal):
    """."""
    x_cal = _calc_poly(x_uncal, y_uncal)
    y_cal = _calc_poly(y_uncal, x_uncal)
    return x_cal, y_cal

def _calc_poly(th1, ot1):
    """."""
    ot2 = ot1*ot1
    ot4 = ot2*ot2
    ot6 = ot4*ot2
    ot8 = ot4*ot4
    th2 = th1*th1
    th3 = th2*th1
    th5 = th3*th2
    th7 = th5*th2
    th9 = th7*th2
    pol = 1e-9 * np.array([
        8.57433100e+06,  4.72784700e+06,  4.03599000e+06,  2.81406000e+06,
        9.67341100e+06,  4.01543800e+06,  1.05648850e+07,  9.85821200e+06,
        8.68409560e+07,  3.94657800e+06,  5.27686400e+06,  2.28461777e+08,
       -1.13979600e+06,  9.54919660e+07,  2.43619500e+07])

    return (
        th1*(pol[0] + ot2*pol[1] + ot4*pol[2] + ot6*pol[3] + ot8*pol[4]) +
        th3*(pol[5] + ot2*pol[6] + ot4*pol[7] + ot6*pol[8]) +
        th5*(pol[9] + ot2*pol[10] + ot4*pol[11]) +
        th7*(pol[12] + ot2*pol[13]) +
        th9*pol[14])

def calc_bpm_pos(x, y):
    R0 = 12e-3
    angant = 6e-3/R0
    phi = np.array([1, 3, 5, 7])*np.pi/4
    phi = np.expand_dims(phi, axis=tuple([i+1 for i in range(x.ndim)]))
    
    d = np.sqrt(x*x + y*y)[None, ...] / R0
    theta = np.arctan2(y, x)[None, ...]
    
    d2 = d*d
    
    sup_esq, sup_dir, inf_dir, inf_esq = np.arctan2(
        (1-d2)*np.sin(angant/2),
        (1+d2)*np.cos(angant/2) - 2*d*np.cos(theta - phi))
    
    sum1 = sup_esq + inf_dir
    sum2 = inf_esq + sup_dir
    dif1 = sup_esq - inf_dir
    dif2 = inf_esq - sup_dir
    
    x_uncal = (dif1/sum1 + dif2/sum2)/2 # /np.sqrt(2)/np.sin(nplenant)*nplenant
    y_uncal = (dif1/sum1 - dif2/sum2)/2 # /np.sqrt(2)/np.sin(nplenant)*nplenant
    
#     x_est, y_est = _apply_linearxy(x_uncal, y_uncal)
    x_est, y_est = _apply_polyxy(x_uncal, y_uncal)
    return x_est, y_est

def simulate_sofb(x0, xl0, y0=0, yl0=0, delta=0, errx=1e-3, erry=1e-3):
    npart = 1000
    bun = pyaccel.tracking.generate_bunch(3.5e-9, 35e-12, 1e-3, 5e-3, twi[0], npart, cutoff=6)
    bun += np.array([x0, xl0, y0, yl0, delta, 0])[:, None]
    rout, *_ = pyaccel.tracking.linepass(ring, bun, indices=bpms)
   
    trajx, trajy = rout[0], rout[2]
    trajx, trajy = calc_bpm_pos(trajx, trajy)

    nanx = np.isnan(trajx)
    snan = np.sum(~nanx, axis=0)
    indcs = snan > npart/3
    trajx = np.nanmean(trajx[:, indcs], axis=0)
    trajy = np.nanmean(trajy[:, indcs], axis=0)
    
    trajx += errx*(np.random.rand(trajx.size)-0.5)*2
    trajy += erry*(np.random.rand(trajy.size)-0.5)*2
    return trajx, trajy, snan


def calc_traj(x0, xl0, y0=0, yl0=0, delta=0, size=160):
    rin = np.array([x0, xl0, y0, yl0, delta, 0])
    rout, *_ = pyaccel.tracking.linepass(ring, rin, bpms[:size])
    return rout[0, :], rout[2, :]

def calc_residue(vec, tx_meas, ty_meas):
    tx_mod, ty_mod = calc_traj(*vec, size=tx_meas.size)
    nanidcs = np.logical_or(np.isnan(tx_mod), np.isnan(tx_mod))
    tx_mod[nanidcs] = 10.0
    ty_mod[nanidcs] = 10.0
    return np.hstack([tx_mod-tx_meas, ty_mod-ty_meas])

def calc_chisqr(residue):
    return np.sum(residue*residue)/residue.size

def calc_jacobian(vec, size=160):
    mat = np.zeros((2*size, 5))
    zer = np.zeros(size)
    dx = 1e-5
    for i in range(vec.size):
        dvec = np.array(vec)
        dvec[i] += dx/2
        res_pos = calc_residue(dvec, zer, zer)
        dvec[i] -= dx
        res_neg = calc_residue(dvec, zer, zer)
        mat[:, i] = (res_pos - res_neg)/dx
    return mat

def calc_init_vals(trajx, trajy):
    x0 = 0
    y0 = 0
    xl0 = 0
    yl0 = 0
    delta = np.mean(trajx) / etax_ave
    return np.array([x0, xl0, y0, yl0, delta])

def do_fitting(vec0, trajx, trajy, max_iter=10, tol=1e-6):
    trax = np.array(trajx)
    tray = np.array(trajy)
    res0 = calc_residue(vec0, trajx, trajy)
    chi0 = calc_chisqr(res0)

    mat = calc_jacobian(vec0, size=trajx.size)
    u, s, vh = np.linalg.svd(mat, full_matrices=False)
    imat = vh.T @ np.diag(1/s) @ u.T
    
    vec, res = vec0.copy(), res0.copy()
    factor = 1
    for _ in range(max_iter):
        dpos = imat @ res
        vec -= dpos * factor
        res = calc_residue(vec, trajx, trajy)
        chi = calc_chisqr(res)
        if chi >= chi0:
            vec = vec0.copy()
            res = res0.copy()
            factor = max(factor/2, 1e-2)
        else:
            vec0 = vec.copy()
            res0 = res.copy()
            chi0 = chi
            factor = min(factor*2, 1)
        if chi0 < tol:
            break
    return vec0
   

In [131]:
np.random.seed(42)
trajx, trajy, trajsum = simulate_sofb(-9e-3, 0.35e-3, 0.0e-3, 0.0, delta=0.01)
print(trajx.size)
vec0 = calc_init_vals(trajx, trajy)
print(vec0)
vecs = [vec0, ]
for i in range(5):
    vec = do_fitting(vecs[-1], trajx, trajy, tol=1e-8)
    print(vec)
    vecs.append(vec)

31
[ 0.          0.          0.          0.         -0.00333179]
[-8.89151985e-03  3.41621211e-04  1.92749040e-04  3.43121261e-05
  8.83531240e-03]
[-9.00816349e-03  3.41569037e-04  2.39221742e-04  3.00517604e-05
  8.97690502e-03]
[-9.01142099e-03  3.41570716e-04  2.39911264e-04  3.01097917e-05
  8.98779599e-03]
[-9.01144492e-03  3.41571743e-04  2.39926271e-04  3.01082361e-05
  8.98755401e-03]
[-9.01144492e-03  3.41571743e-04  2.39926271e-04  3.01082361e-05
  8.98755401e-03]


In [132]:
f  = plt.figure(figsize=(9, 10))
gs = mpl_gs.GridSpec(3, 1)
gs.update(left=0.12, right=0.98, top=0.97, bottom=0.10, hspace=0.15)
ax = plt.subplot(gs[0, 0])
ay = plt.subplot(gs[1, 0])
asum = plt.subplot(gs[2, 0])

ax.plot(bpmpos[:trajx.size], 1e3*trajx, label='Trajectory')
ay.plot(bpmpos[:trajy.size], 1e3*trajy)
asum.plot(bpmpos[:trajsum.size], trajsum)
for i, vec in enumerate(vecs):
    trajx_fit, trajy_fit = calc_traj(*vec, size=trajx.size)
    ax.plot(bpmpos[:trajx_fit.size], 1e3*trajx_fit, '-o', label='Fit {:d}'.format(i), linewidth=1)
    ay.plot(bpmpos[:trajy_fit.size], 1e3*trajy_fit, '-o', linewidth=1)
ax.legend()
asum.set_xlabel('Position [m]')
ax.set_ylabel('X [mm]')
ay.set_ylabel('Y [mm]')
asum.set_ylabel('Sum [counts]')
f.show()

In [106]:
f  = plt.figure(figsize=(9, 7))
gs = mpl_gs.GridSpec(2, 1)
gs.update(left=0.12, right=0.98, top=0.97, bottom=0.10, hspace=0.15)
ax = plt.subplot(gs[0, 0])
ay = plt.subplot(gs[1, 0])

x = np.linspace(0, 11.9e-3, 100)
y = np.linspace(0, 11.9e-3, 100)*0
x_est, y_est = calc_bpm_pos(x, y)
ax.plot(x, x_est, '-o')
ay.plot(y, y_est, '-o')
ax.legend()
ay.set_xlabel('Position [m]')
ax.set_ylabel('X [mm]')
ay.set_ylabel('Y [mm]')
f.show()

No handles with labels found to put in legend.
